<a href="https://colab.research.google.com/github/yunhsuan1009/yunhsuan/blob/main/EX04_03_%E5%AE%A2%E6%88%B6%E5%88%86%E7%BE%A4ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義
將 性別 欄位 Label Encoding
將客戶分成 3 群，並解讀各群特質
計算 k = 2 ~ 15 的 Calinski-Harbasz Score，找出最佳 k 值

## 資料收集

In [ ]:
!wget -O car_models.csv https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv

--2025-05-27 07:36:33--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2839 (2.8K) [text/plain]
Saving to: ‘car_models.csv’

car_models.csv      100%[===================>]   2.77K  --.-KB/s    in 0s      

2025-05-27 07:36:33 (29.5 MB/s) - ‘car_models.csv’ saved [2839/2839]



In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'
df = pd.read_csv(url)
df.head()

,性別,年齡,收入（千）,消費指數（1~100）
0,女,74,38,81
1,女,51,71,91
2,女,30,65,10
3,女,88,49,17
4,女,55,48,70


## 資料前處理

### 資料清理

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   性別           200 non-null    object
 1   年齡           200 non-null    int64 
 2   收入（千）        200 non-null    int64 
 3   消費指數（1~100）  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.4+ KB


### 探索性分析

In [ ]:
df_cor = df.drop(columns='性別').corr()
df_cor

,年齡,收入（千）,消費指數（1~100）
年齡,1.000000,0.031519,-0.127454
收入（千）,0.031519,1.000000,0.031476
消費指數（1~100）,-0.127454,0.031476,1.000000


### 資料分割

### 類別轉換

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['女','男']])
df[['性別']] = encoder.fit_transform(df[['性別']])
df

,性別,年齡,收入（千）,消費指數（1~100）
0,0.0,74,38,81
1,0.0,51,71,91
2,0.0,30,65,10
3,0.0,88,49,17
4,0.0,55,48,70
...,...,...,...,...
195,1.0,86,84,82
196,1.0,59,52,30
197,0.0,63,29,61
198,1.0,67,80,9


### 特徵縮放

## 模型訓練

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

KMeans(n_clusters=3)

In [ ]:
df['cluster'] = kmeans.labels_
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,0.0,74,38,81,0
1,0.0,51,71,91,1
2,0.0,30,65,10,2
3,0.0,88,49,17,2
4,0.0,55,48,70,0
...,...,...,...,...,...
195,1.0,86,84,82,1
196,1.0,59,52,30,2
197,0.0,63,29,61,0
198,1.0,67,80,9,2


## 模型評估

In [ ]:
df.groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.515625,48.796875,34.87500,61.562500
1,0.625000,49.828125,74.84375,79.734375
2,0.569444,59.027778,65.87500,20.402778


In [ ]:
from sklearn.metrics import calinski_harabasz_score
score = calinski_harabasz_score(df.drop(columns='cluster'), kmeans.labels_)
score

np.float64(95.36808392176823)

## 模型調整

In [ ]:
df_nocluster = df.drop(columns='cluster')

for i in range(2,16):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(df)
  score = calinski_harabasz_score(df.drop(columns='cluster'), kmeans.labels_)
  print(f'k={i} score={score}')

k=2 score=111.68583137120146
k=3 score=96.58085896888944
k=4 score=86.70516770460966
k=5 score=86.01971150944375
k=6 score=77.31134563620873
k=7 score=94.05921000928736
k=8 score=88.05387353100649
k=9 score=95.6405599761582
k=10 score=93.92261910880853
k=11 score=93.27905817586785
k=12 score=89.828854168514
k=13 score=83.15251911331161
k=14 score=85.65818616854703
k=15 score=86.183145060428


## 模型部署

### 儲存模型

### 推論預測